# Meta-heuristieken

In [1]:
import math
import numpy as np
import pandas as pd
from simanneal import Annealer

## Vraag 1: TSP nearest neighbour benadering
Slide 14: Geef de oplossing indien je start bij punt b.

## Vraag 2: De rugzak (aka the knapsack problem)
Je bevindt je in een geheime kamer die uitgerust is met een deur met tijdslot. Je ziet een timer aftellen die meldt dat je nog maar vijf minuten over het alvorens de deur voor altijd op slot zal zijn. Voor je neus liggen waardevolle voorwerpen met elk hun eigen opbrengst en gewicht. Je hebt een rugzak bij die een absoluut maximaal gewicht kan torsen van 750gr.   Op Canvas vind je de lijst van voorwerpen met hun gewicht en opbrengst.  Stel de optimale rugzak samen.  Je zou op een optimale opbrengst van 1458 moeten uitkomen (of toch zeker een waarde dicht daarbij in de buurt).

In [2]:
knapsack_items = pd.read_csv('../Datasets/Knapsack Items.csv', delimiter=',')
knapsack_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Unnamed: 0     15 non-null     object
 1   gewichten(gr)  15 non-null     int64 
 2   waarde         15 non-null     int64 
dtypes: int64(2), object(1)
memory usage: 488.0+ bytes


In [3]:
class KnapSackProblem(Annealer):
    def energy(self):
        oplossing = self.state
        totaalgewicht = (oplossing * gewichten).sum()
        if totaalgewicht > 750:
            totaal = 0
        else:
            totaal = (oplossing * waarde).sum()
        
        return -totaal
    
gewichten = knapsack_items['gewichten(gr)']
waarde = knapsack_items['waarde']

solution = np.random.randint(0,2, size=len(knapsack_items))
probleem = KnapSackProblem(solution)
probleem.anneal()

 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     2.50000      -1382.00   100.00%     0.00%     0:00:04     0:00:00

(array([0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1]), -1382)

 ## Vraag 3: De dakgoten
Je bent belast met het ontwerp van dakgoten waarbij de productiekost zo laag mogelijk moet zijn. Daarom is het noodzakelijk dat de dakgoten een zo optimale doorsnede hebben met het beschikbare materiaal zodat bladeren en vuil makkelijk afgevoerd kunnen worden.  Het bedrijf waarvoor je werkt koopt metalen platen aan die een breedte hebben van 1m. M.a.w. h + b + h  -zie tekening- moet kleiner of gelijk zijn aan 1m.  Bepaal de ideale breedte B en hoogte H van de dakgoot die je uit de platen van 1m kan maken.

```
  |       |
h |       |
  |_______|
      b
```


In [4]:
class Dakpannen(Annealer):
    def energy(self):
        b = self.state[0]
        h = (1 - b)/2
        
        return -b*h # - gebruiken want kan max 1 m zijn 
    
    def move(self):
        self.state[0] += np.random.normal(0,0,1)
        self.state[0] = np.clip(self.state[0], 0, 1) #dit kan max 1 zijn dus 
        return

solution = np.random.uniform(0,1, size=1)
dakpan = Dakpannen(solution)
dakpan.anneal()
        

 Temperature        Energy    Accept   Improve     Elapsed   Remaining
/var/folders/4n/z403ds5x0n141m695xdfdlcm0000gn/T/ipykernel_11321/247075166.py:9: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  self.state[0] += np.random.normal(0,0,1)
     2.50000         -0.01   100.00%     0.00%     0:00:01     0:00:00

(array([0.98535764]), -0.0072139827639136145)

## Vraag 4: Voetbalstadium
De plaatselijke sportclub wil een nieuw stadion bouwen.  Het sportveld bestaat uit een rechthoek met 2 halve cirkels rechts en links, zie figuur. De omtrek moet 400m bedragen. Tegelijkertijd willen we ervoor zorgen dat het centrale middenveld (de rechthoek) een maximale oppervlakte heeft.   Bepaal de ideale lengte –en breedteverhouding.

```
                  rechthoek
                 ___________
              / |           | \
halve cirkel |  |B          |  | halve cirkel
              \ |___________| /
                      L
```

In [7]:
class SoccerField(Annealer):
    def energy(self):
        L = self.state[0]
        B = (400 - 2 * L)/math.pi
        
        return -L*B
    
    def move(self):
        self.state[0] += np.random.normal(0,0,1)
        self.state[0] = np.clip(self.state[0], 0, 200) #helf van veld 
        return

solution = np.random.uniform(0,200, size=1)
probleem = SoccerField(solution)
probleem.anneal()

 Temperature        Energy    Accept   Improve     Elapsed   Remaining
/var/folders/4n/z403ds5x0n141m695xdfdlcm0000gn/T/ipykernel_11321/1328682734.py:9: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  self.state[0] += np.random.normal(0,0,1)
     2.50000      -2594.86   100.00%     0.00%     0:00:01     0:00:00

(array([23.03243962]), -2594.8587852779706)

## Vraag 5: Optimalisatie
Gegeven volgende te maximaliseren doelfunctie:

obj = 0.2 + x^2 + y^2 - 0.1 cos(6*pi*x) - 0.1 cos(6*pi*y)

Met volgende beperkingen:  -1.0 ≤ x ≤  1.0	en -1.0 ≤ y ≤  1.0
Zoek een goede oplossing.


In [15]:
class OptimalisatieProbleem(Annealer):
    def energy(self):
        x = self.state[0]
        y = self.state[1]
        
        return -(0.2 + x**2 + y**2 - 0.1 * math.cos(6*math.pi*x) - 0.1 * math.cos(6*math.pi*y))

        
    def move(self):
        i = np.random.randint(0,2)
        self.state[i] += np.random.normal(0,0.1)
        self.state[i] = np.clip(self.state[i], -1, 1)

In [16]:
solution = np.random.uniform(-1, 1, size=2)
probleem = OptimalisatieProbleem(solution)
probleem.anneal()

 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     2.50000         -1.46    98.20%    48.00%     0:00:01     0:00:00

(array([-1.,  1.]), -2.0)